In [ ]:
import ciao_contrib.runtool as rt
import os
import pandas as pd
import astropy.io.fits as fits
import pyds9 as ds9

In [ ]:
start = int(input('From which source to start '))
n = int(input('How many sources '))
print('#==============================================#')
#Creating a CSV file for monitoring the source no and obsid which are acis
acis_src = open('acis_sources.csv', 'a')
prob_src = open('data_red_problematic.txt','a')
acis_src.write('source_no.,obsid\n')

#Data reduction (Chandra_repro(on all observations) and making regions(only for acis observations))
counts = 0
for i in range(start, start+n):
    
    #ra dec of source i
    with open(f'source{i}/details.txt') as details:
        lines = details.readlines()
        ra = lines[1][5:-1]
        dec = lines[2][6:-1]
    
    #obsids of source i
    obsids = pd.read_csv(f'source{i}/obs_info.csv', sep = ',',names = ['obsid'])['obsid']
    for obsid in obsids:
        counts += 1
        flag = 0
        print(f'{counts}\tsource{i}\t{obsid} ')
       
        #Rename Primary files in obsid
        os.system(f'ls source{i}/{obsid}/primary/*.gz > source{i}/{obsid}/primary/gz_list')
        filenames = open(f'source{i}/{obsid}/primary/gz_list').readlines()
        for filename in filenames:
            my_source = filename[:-1]
            new_name = filename[:-4]
            my_dest = new_name
            # rename() function will
            # rename all the files
            os.rename(my_source, my_dest)

        #Rename Secondary files
        os.system(f'ls source{i}/{obsid}/secondary/*.gz > source{i}/{obsid}/secondary/gz_list')
        filenames = open(f'source{i}/{obsid}/secondary/gz_list').readlines()
        for filename in filenames:
            my_source = filename[:-1]
            new_name = filename[:-4]
            my_dest = new_name
            # rename() function will
            # rename all the files
            os.rename(my_source, my_dest)
        
        try :
            # Running chandra repro for the given obsid 
            print('\t\treprocessing')
            flag = 1
            rt.chandra_repro(f'source{i}/{obsid}/')
            print('\t\treprocessing done')
            
            # Making regions in repro folder
            repro_path = f'source{i}/{obsid}/repro/'
            os.system(f'ls {repro_path}*repro_evt2.fits > {repro_path}repro_evt.txt')
            evt_filename = open(f'{repro_path}repro_evt.txt').readlines()[0][len(repro_path):-1] #reprocessed event file name

            if evt_filename[:4] != 'acis' :
                #if not an acis observation then left it
                print('\t\tnot acis')
                flag = 2
                continue

            else :
                #Adding this observation to the acis files
                acis_src.write(f'source{i},{obsid}\n')

                #Creating psf
                print('\t\tcreating psf')
                ecf = 0.92 #psf fraction
                rt.psfsize_srcs.infile = f'{repro_path}{evt_filename}'
                rt.psfsize_srcs.pos = f'{ra} {dec}'
                rt.psfsize_srcs.outfile = f'{repro_path}psf_size_{ecf}.fits'
                rt.psfsize_srcs.energy = 'broad'
                rt.psfsize_srcs.ecf = ecf
                rt.psfsize_srcs.clobber = 'yes'
                rt.psfsize_srcs()

                psf = fits.open(f'{repro_path}psf_size_{ecf}.fits')[1].data
                shape = psf['SHAPE'][0]
                x = psf['X'][0]
                y = psf['Y'][0]
                r = psf['R'][0]
                
                print('\t\tcreting regions')
                #creating source region
                src_reg = open(f'{repro_path}source_region.reg', 'w')
                src_reg.write(f'{shape}({x},{y},{r})')
                src_reg.close()

                #creating background region
                bkg_reg = open(f'{repro_path}background_region.reg', 'w')
                bkg_reg.write(f'annulus({x},{y},{1.2*r},{2*r})')
                bkg_reg.close()

                #Place to save the images of the region for checking
#                 os.system('mkdir regions')
                
                print('\t\tshowing regions in ds9')
                #Interaction with ds9
                d = ds9.DS9(wait=60)
                d.set(f"file {repro_path}{evt_filename}")
                d.set(f"bin buffersize 8192")
                d.set(f"region {repro_path}source_region.reg")
                d.set(f"region {repro_path}background_region.reg")
                d.set(f"crop {int(x)} {int(y)} {int(10*r)} {int(10*r)}")
                d.set("zoom to fit")
                d.set(f"saveimage {repro_path}regions.jpg")
                d.set(f"saveimage regions/source{i}_{obsid}_regions.jpg")
                d.set('quit')
                print('\t\tDone :-)')
                print('\n---------------------------------------------\n')
            
        except:
            if flag == 1:
                str = f'(error : {flag}) Processing Error'
            if flag == 2:
                str =  f'(error : {flag}) Not ACIS hence'
                
            prob_src.write(f'source{i} {obsid} {str} not done :-(\n')
            print(f'\t\t{str} not done :-(')
            print('\n---------------------------------------------\n')
            continue
            
prob_src.close()
acis_src.close()          